In [14]:
import numpy as np 
import xarray as xr 
import torch

import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
ozone_2011 = xr.open_dataset("/Volumes/Extreme SSD/PRL/data/high_res/WRF_2011.nc")
co_no2_2011 = xr.open_dataset("/Volumes/Extreme SSD/PRL/data/high_res/WRF_Archi_2011_CO_NO2.nc")
no_2011 = xr.open_dataset("/Volumes/Extreme SSD/PRL/data/high_res/WRF_Archi_2011_NO.nc")
humidity_2011 = xr.open_dataset("/Volumes/Extreme SSD/PRL/data/high_res/WRF_Archi_2011_SpecificHum.nc")
temp_2011 = xr.open_dataset("/Volumes/Extreme SSD/PRL/data/high_res/WRF_2011_Archi_T.nc")
TIME_POINTS, PRESSURE_POINTS, LAT_POINTS, LON_POINTS = ozone_2011["o3"].shape

combined_data = xr.Dataset(
    {
        "ozone": ozone_2011["o3"],
        "pm25": ozone_2011["PM2_5_DRY"],
        "co":  co_no2_2011["co"],
        "no2": co_no2_2011["no2"],
        "no": no_2011["no"],
        "humidity": humidity_2011["QVAPOR"],
        "temperature": temp_2011["T2"].expand_dims({"bottom_top": np.arange(PRESSURE_POINTS)}, axis=1)
    },
    coords={"time": ozone_2011["Times"], 
            "pressure": ozone_2011["bottom_top"], 
            "latitude": ozone_2011["south_north"], 
            "longitude": ozone_2011["west_east"]})

In [21]:
data = combined_data["ozone"][0, 0, :, :].values

def image_to_chunks(image, chunk_size=(16, 16)):
    image_height, image_width = image.shape[:2]
    chunk_height, chunk_width = chunk_size

    # Initialize a list to store the chunks
    chunks = []

    # Iterate over the image and extract chunks
    for y in range(0, image_height, chunk_height):
        for x in range(0, image_width, chunk_width):
            chunk = image[y:y + chunk_height, x:x + chunk_width]
            chunks.append(chunk)

    return chunks


def plot_image_chunks(image, chunk_size=(16, 16), border_size=2):
    chunks = image_to_chunks(image, chunk_size)
    chunk_height, chunk_width = chunk_size

    # Calculate the number of chunks along each dimension
    num_chunks_y = (image.shape[0] + chunk_height - 1) // chunk_height
    num_chunks_x = (image.shape[1] + chunk_width - 1) // chunk_width

    # Create a figure with subplots
    fig, axes = plt.subplots(num_chunks_y, num_chunks_x, figsize=(num_chunks_x * 2, num_chunks_y * 2))

    # Plot each chunk
    for i in range(num_chunks_y):
        for j in range(num_chunks_x):
            ax = axes[i, j]
            chunk_index = i * num_chunks_x + j
            if chunk_index < len(chunks):
                ax.imshow(chunks[chunk_index])
            ax.axis('off')

    # Adjust layout to add borders
    plt.subplots_adjust(wspace=border_size / chunk_width, hspace=border_size / chunk_height)
    plt.show()

chunks = image_to_chunks(data, chunk_size=(16, 16))

In [21]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, use_activation: bool, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size, stride, padding, bias = True)
        self.activation = nn.LeakyReLU(0.2, inplace=True) if use_activation else nn.Identity()

    def forward(self, x):
        return self.activation(self.conv(x))
    
class DenseResidualBlock(nn.Module):
    def __init__(self, in_channels: int, channels = 32, beta: float = 0.2, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.beta = beta 
        self.conv = nn.ModuleList()

        for block_no in range(5):
            self.conv.append(ConvBlock(in_channels + channels * block_no, 
                                       channels if block_no < 4 else in_channels,
                                         use_activation=True if block_no < 4 else False,
                                           kernel_size=3, stride=1, padding=1))
            
            
    def forward(self, x):
        new_inputs = x
        for block in self.conv:
            out = block(new_inputs)
            new_inputs = torch.cat([new_inputs, out], dim=1)
        return self.beta * out + x
    

class RRDB(nn.Module):
    def __init__(self, in_channels, residual_beta, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.residual_beta = residual_beta
        self.rrdb = nn.Sequential(*[DenseResidualBlock(in_channels, beta=residual_beta) for _ in range(3)])

    def forward(self, x):
        return self.rrdb(x) * self.residual_beta + x
    

class ModifiedSRCNN(nn.Module):
    def __init__(self, in_channels: int, num_blocks: int, 
                 n1: int, n2: int, f1: int, f2: int, f3: int,
                 *args, **kwargs) -> None:
        """ Initialize the SRCNN with Dense Residual network model with the required layers 
         Below params are the hyperparameters for the SRCNN model without the 
         Bassic block which has been added extra other than the resisual connections.
        in_channels (int): Input number of channels
        num_blocks (int): Number of RRDB blocks
        n1 (int): Number of filters in the first convolutional layer
        n2 (int): Number of filters in the second convolutional layer
        f1 (int): Kernel size of the first convolutional layer
        f2 (int): Kernel size of the second convolutional layer
        f3 (int): Kernel size of the third convolutional layer
        residual_beta (float): Residual connection weight
        """
        super().__init__(*args, **kwargs)
        self.conv1 = ConvBlock(in_channels, n1, kernel_size=f1, stride=1, padding=(1, 1, 1), use_activation=True)
        self.blocks = nn.Sequential(*[RRDB(n1 + in_channels, residual_beta=0.5) for _ in range(num_blocks)])
        self.conv2 = ConvBlock(2 * (n1 + in_channels), n2, kernel_size=f2, stride=1, padding=(1, 1, 1), use_activation=True)
        self.conv3 = ConvBlock(n2 + n1 + in_channels, in_channels, kernel_size=f3, stride=1, padding=1, use_activation=False)
        self._initialize_weights()

    def forward(self, x):
        initial = x 
        print(f"Initial shape: {initial.shape}")
        x = self.conv1(x)
        x = torch.concat([x, initial], dim = 1)
        print(f"Conv1 shape: {x.shape}")
        initial = x
        print(f"Initial shape: {initial.shape}")
        x = self.blocks(x)
        x = torch.concat([x, initial], dim = 1)
        print(f"Blocks shape: {x.shape}")
        x = self.conv2(x)
        print(f"shape after conv 2: {x.shape}")
        x = torch.concat([x, initial], dim=1)
        print(f"concatenated shape : {x.shape}")
        x = self.conv3(x)
        print(f"Final shape = {x.shape}")
        return x 
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                torch.nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    torch.nn.init.zeros_(m.bias)


In [22]:
random_tensor = torch.rand(32, 7, 50, 20, 20, dtype=torch.float32).to(DEVICE)

in_channels = 7
num_blocks = 3
n1 = 32
n2 = 64
f1 = 3
f2 = 3
f3 = 3
srcnn = ModifiedSRCNN(in_channels = in_channels, num_blocks = num_blocks, n1 = n1, n2 = n2, f1 = f1, f2 = f2, f3 = f3).to(DEVICE)